In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
train_data = pd.read_csv('/content/drive/MyDrive/KT_aivle/딥러닝/RNN/미니프로젝트/train.csv')

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.6 MB/s eta 0:00:00


In [4]:
import numpy as np

In [5]:
label_dict = {
        "코드1" : 0,
        "코드2" : 0,
        "웹" : 1,
        "이론" : 2,
        "시스템 운영" : 3,
        "원격" : 4
    }

# 레이블을 숫자로 변환
train_data["label"] = train_data["label"].map(label_dict)

In [15]:
!pip install transformers
import tensorflow as tf
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# 데이터 로드 및 전처리
X_train = train_data["text"].values
y_train = train_data["label"].values
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
model_name = "beomi/kcbert-base"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = TFElectraForSequenceClassification.from_pretrained(model_name, from_pt=True, num_labels=5)
# 입력 데이터 전처리 함수 정의
def preprocess(texts):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            pad_to_max_length=True,
            return_attention_mask=True,
        )
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])
    return np.array(input_ids), np.array(attention_masks)
# 입력 데이터 전처리
X_train_ids, X_train_masks = preprocess(x_train)
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_val_one_hot = tf.keras.utils.to_categorical(y_val)
# 모델 학습
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.fit(
    {"input_ids": X_train_ids, "attention_mask": X_train_masks},
    {"output_0": y_train_one_hot},
    epochs=10,
    batch_size=32,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ElectraTokenizer'.
You are using a model of type bert to instantiate a model of type electra. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.1.intermediate.dense.weight', 'bert.encoder.layer.2.attention.self.query.bias', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.pooler.dense.bias', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.5.output.LayerNorm.weight', 'bert.encoder.layer.7.intermediate.dense.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.4.atte

Epoch 1/10
105/105 [==============================] - 125s 695ms/step - loss: 8.8777 - accuracy: 0.1919
Epoch 2/10
105/105 [==============================] - 74s 704ms/step - loss: 7.0610 - accuracy: 0.1988
Epoch 3/10
105/105 [==============================] - 75s 710ms/step - loss: 7.0079 - accuracy: 0.1988
Epoch 4/10
105/105 [==============================] - 75s 716ms/step - loss: 6.7711 - accuracy: 0.1988
Epoch 5/10
105/105 [==============================] - 75s 717ms/step - loss: 7.1529 - accuracy: 0.1988
Epoch 6/10
105/105 [==============================] - 75s 716ms/step - loss: 7.0127 - accuracy: 0.1988
Epoch 7/10
105/105 [==============================] - 75s 717ms/step - loss: 7.0804 - accuracy: 0.1988
Epoch 8/10
105/105 [==============================] - 75s 719ms/step - loss: 7.0320 - accuracy: 0.1988
Epoch 9/10
105/105 [==============================] - 71s 678ms/step - loss: nan - accuracy: 0.4045
Epoch 10/10
105/105 [==============================] - 71s 675ms/step - los

In [16]:
# validation 데이터 전처리
X_val_ids, X_val_masks = preprocess(x_val)

# 모델 예측
y_pred_one_hot = model.predict({"input_ids": X_val_ids, "attention_mask": X_val_masks})["logits"]
y_pred = tf.argmax(y_pred_one_hot, axis=1)

12/12 [==============================] - 6s 247ms/step


In [17]:
y_pred

<tf.Tensor: shape=(371,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [18]:
from sklearn.metrics import *
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.44      1.00      0.61       164
           1       0.00      0.00      0.00        69
           2       0.00      0.00      0.00        78
           3       0.00      0.00      0.00        47
           4       0.00      0.00      0.00        13

    accuracy                           0.44       371
   macro avg       0.09      0.20      0.12       371
weighted avg       0.20      0.44      0.27       371



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
